## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
filetoload = "..\Weather_Database\WeatherPy_Database.csv" 
city_data_df = pd.read_csv(filetoload)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,punta arenas,CL,-53.1500,-70.9167,35.71,93,0,5.75,clear sky
1,1,carnarvon,AU,-24.8667,113.6333,71.67,57,40,10.36,scattered clouds
2,2,longyearbyen,SJ,78.2186,15.6401,37.24,65,75,5.75,broken clouds
3,3,mataura,NZ,-46.1927,168.8643,49.53,80,39,3.09,scattered clouds
4,4,nikolskoye,RU,59.7035,30.7861,64.40,90,100,5.01,overcast clouds


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature for your trip destination ?  "))
max_temp = float(input("What is the maximum temperature for your trip destination ?  "))

What is the minimum temperature for your trip destination ?  70
What is the maximum temperature for your trip destination ?  85


In [5]:
print(min_temp,max_temp)

70.0 85.0


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
trip_dest_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                 (city_data_df["Max Temp"] <= max_temp)]
trip_dest_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,carnarvon,AU,-24.8667,113.6333,71.67,57,40,10.36,scattered clouds
6,6,atuona,PF,-9.8000,-139.0333,79.65,74,38,16.62,light rain
11,11,luanda,AO,-8.8368,13.2343,75.20,94,75,3.44,broken clouds
21,21,boundiali,CI,9.5217,-6.4869,73.24,81,68,3.22,broken clouds
24,24,namie,JP,37.4833,141.0000,83.55,64,57,1.99,broken clouds
25,25,cayenne,GF,4.9333,-52.3333,75.24,91,75,0.00,broken clouds
26,26,sakakah,SA,29.9697,40.2064,80.17,21,0,8.48,clear sky
30,30,matara,LK,5.9485,80.5353,83.52,76,87,13.58,overcast clouds
32,32,eyl,SO,7.9803,49.8164,80.01,61,75,25.43,broken clouds
35,35,bambous virieux,MU,-20.3428,57.7575,75.45,73,75,13.80,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
trip_dest_df.count()

City_ID                263
City                   263
Country                263
Lat                    263
Lng                    263
Max Temp               263
Humidity               263
Cloudiness             263
Wind Speed             263
Current Description    263
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = trip_dest_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,carnarvon,AU,71.67,scattered clouds,-24.8667,113.6333,
6,atuona,PF,79.65,light rain,-9.8000,-139.0333,
11,luanda,AO,75.20,broken clouds,-8.8368,13.2343,
21,boundiali,CI,73.24,broken clouds,9.5217,-6.4869,
24,namie,JP,83.55,broken clouds,37.4833,141.0000,
25,cayenne,GF,75.24,broken clouds,4.9333,-52.3333,
26,sakakah,SA,80.17,clear sky,29.9697,40.2064,
30,matara,LK,83.52,overcast clouds,5.9485,80.5353,
32,eyl,SO,80.01,broken clouds,7.9803,49.8164,
35,bambous virieux,MU,75.45,broken clouds,-20.3428,57.7575,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        # Grab the first hotel from the results and store the name.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found .. Skipping")
        continue      

Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping
Hotel not found .. Skipping


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,carnarvon,AU,71.67,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
6,atuona,PF,79.65,light rain,-9.8000,-139.0333,Villa Enata
11,luanda,AO,75.20,broken clouds,-8.8368,13.2343,LANDU MARINEZA LUFUANDU TONI
21,boundiali,CI,73.24,broken clouds,9.5217,-6.4869,Hotel Paysan Boundiali
24,namie,JP,83.55,broken clouds,37.4833,141.0000,いこいの村なみえ
25,cayenne,GF,75.24,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
26,sakakah,SA,80.17,clear sky,29.9697,40.2064,Fakhamat Al Orjoana Apartment
30,matara,LK,83.52,overcast clouds,5.9485,80.5353,Amaloh Boutique Resort
32,eyl,SO,80.01,broken clouds,7.9803,49.8164,Amiin Hotel
35,bambous virieux,MU,75.45,broken clouds,-20.3428,57.7575,Casa Tia Villa


In [12]:
hotel_df.count()

City                   263
Country                263
Max Temp               263
Current Description    263
Lat                    263
Lng                    263
Hotel Name             263
dtype: int64

In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
# 11b. Display the figure
fig = gmaps.figure(center=(30.1,31.1),zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, max_temp,dissipating=False,max_intensity=300,point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))